<a href="https://colab.research.google.com/github/Ubaid-Manzoor/CatVsDog_ImageClassifier/blob/master/DogVsCat_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#This Block Of Code Only Needed in Google Colab

from google.colab import files
files.upload()

In [0]:
#This Block Of Code Only Needed in Google Colab

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

#To Avoid Warning
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
#This Block Of Code Only Needed in Google Colab
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../content"))

# Any results you write to the current directory are saved as output.

['.config', 'test', 'sample_submission.csv', 'kaggle.json', 'y_train', 'test_data', 'X_train', 'train', 'dogsVScats.csv', 'test.zip', 'train.zip', 'logs', 'sample_data']


In [0]:
# #This Block Of Code Only Needed in Google Colab

# !unzip "/content/train.zip"

In [0]:
# #This Block Of Code Only Needed in Google Colab

# !unzip "/content/test.zip"

In [0]:
import matplotlib.pyplot as plt
import cv2
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , Activation , Flatten , Conv2D , MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard , ModelCheckpoint
import time
import pickle

In [0]:
#Give Model a Name

name = "CatVsDog"

In [0]:
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(name))

In [0]:
checkpoint = ModelCheckpoint(filepath='BestModel' , save_best_only = True , mode = 'max')

In [0]:
traindir = "train"
testdir = "test"
def get_ytrain(traindir):
    y_train = []
    for img_name in os.listdir(traindir):
        animal_name = []
        animal_name = img_name.split('.')[0]
        y_train.append(0 if animal_name == 'cat' else 1)
    return y_train
def get_data(dir_path="train" , img_size=100 , data_type="train"):
    data = []
    if data_type == "train":
        y_train = get_ytrain(dir_path)
    for index,img in enumerate(os.listdir(dir_path)):
        img_array = cv2.imread(os.path.join(dir_path,img))
        new_array = cv2.resize(img_array , (img_size , img_size))
        if data_type=="train":
            data.append([new_array , y_train[index]])
        else:
            data.append(new_array)
    return data

In [0]:
# training_data = get_data(img_size=100)

In [0]:
# test_data = get_data(dir_path = testdir , img_size=100,data_type="test")
# test_data = np.array(test_data)
# print(test_data.shape)

In [0]:
# X_train = []
# y_train = []

In [0]:
# for feature , label in training_data:
#     X_train.append(feature)
#     y_train.append(label)

In [0]:
# X_train = np.array(X_train)
# y_train = np.array(y_train).reshape(-1,1)
# print(X_train.shape)
# print(y_train.shape)

In [0]:
# #Save Data
# def save_data(file_name , data_to_dump):
#     with open(file_name , mode='wb') as data:
#         pickle.dump(data_to_dump , data)

# save_data("X_train" , X_train)
# save_data("y_train" , y_train)
# save_data("test_data" , test_data)

In [17]:
#Retreive Data Back 
def load_data(file_name):
    with open(file_name,mode='rb') as file:
        data = pickle.load(file)
    return data
X_train = load_data("X_train")
print(X_train.shape)
y_train = load_data("y_train")
print(y_train.shape)
test_data = load_data("test_data")
print(test_data.shape)

(25000, 100, 100, 3)
(25000, 1)
(12500, 100, 100, 3)


In [0]:
X_train = X_train/255.0
test_data = test_data/255.0

In [34]:
#model
model = Sequential()
model.add((Conv2D(filters=32 , kernel_size=(8,8) , input_shape = X_train.shape[1:])))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Dropout(0.3))

model.add((Conv2D(filters=64 , kernel_size=(6,6))))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.3))

# model.add((Conv2D(filters=128 , kernel_size=(4,4))))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

model.add(Flatten())
# model.add(Dense(16))
# model.add(Activation("relu"))
# model.add(Dropout(0.2))

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss='binary_crossentropy',
             optimizer="adam",
             metrics=['accuracy'])
model.fit(X_train , y_train ,epochs=5
          ,  batch_size=32,validation_split=0.2 , callbacks=[tensorboard , checkpoint])

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 24s 1ms/step - loss: 0.6674 - acc: 0.5873 - val_loss: 0.6351 - val_acc: 0.6562
Epoch 2/5
20000/20000 [==============================] - 23s 1ms/step - loss: 0.6156 - acc: 0.6685 - val_loss: 0.5864 - val_acc: 0.7084
Epoch 3/5
20000/20000 [==============================] - 19s 958us/step - loss: 0.5725 - acc: 0.7037 - val_loss: 0.5802 - val_acc: 0.6998
Epoch 4/5
20000/20000 [==============================] - 19s 953us/step - loss: 0.5454 - acc: 0.7238 - val_loss: 0.5252 - val_acc: 0.7450
Epoch 5/5
20000/20000 [==============================] - 20s 1ms/step - loss: 0.5223 - acc: 0.7425 - val_loss: 0.5229 - val_acc: 0.7370


In [0]:
prediction = model.predict(test_data)

In [29]:
prediction.shape

(12500, 1)

In [0]:
prediction = prediction.reshape(prediction.shape[0],)


In [31]:
prediction

array([0.09492346, 0.00331743, 0.9985228 , ..., 0.8251995 , 0.60039186,
       0.3540866 ], dtype=float32)

In [32]:
counter = range(1, len(test_data) + 1)
solution = pd.DataFrame({"id": counter, "label":list(prediction)})
 

solution.to_csv("dogsVScats.csv", index = False)
solution

,id,label
0,1,9.492346e-02
1,2,3.317432e-03
2,3,9.985228e-01
3,4,6.616721e-02
4,5,6.991080e-01
5,6,3.375899e-03
6,7,8.819088e-02
7,8,7.007869e-01
8,9,8.107682e-01
9,10,1.527716e-02
